# <center>CS 207 Final Project: Documentation</center>
<center>December 2019</center>

## Introduction


Efficiently and accurately evaluating derivatives of functions is one of the most important operations in science and engineering. Automatic differentiation (AD) is a technique which, given a function $f$ and a point, automatically evaluates that point's derivative. AD is less costly than symbolic defferentiation, while achieving machine precision compared with finite differentiation. This library implements both forward mode and reverse mode of AD. A user can simply input a function $f$ and a vector $\mathbf{x}$, specify the mode that they want, and then they will be able to get the value and derivative computed by the specified mode right away.


## Background

We present below some of the key concepts and formulae upon which we build the veritorch library:

### Chain rule

Chain rule is fundamental to AD when we decompose functions.

Suppose we have $h(u(t), v(t))$, its derivative with respect to $t$ is:

$$\frac{\partial h}{\partial t} = \frac{\partial h}{\partial u}\frac{\partial u}{\partial t} + \frac{\partial h}{\partial v}\frac{\partial v}{\partial t}.$$

For the general function $h(y_1(\mathbf{x}), \dotsc,y_n(\mathbf{x}))$, where we replace $t$ with a vector $\mathbf{x} \in \mathbb{R}^m$ and $h$ a function of $n$ other functions $y_i$, the derivative is:

$$\nabla_x h = \sum_{i=1}^n \frac{\partial h}{\partial y_i} \nabla y_i(\mathbf{x})$$

### Jacobians and vectors

If we have a function $\mathbf{y}(\mathbf{x})$: $\mathbb{R}^n \rightarrow \mathbb{R}^m$, the Jacobian matrix of it is a matrix representing all the possible partial derivatives combinations as follows:
$$
\mathbf{J} = \begin{bmatrix}
\frac{\partial \mathbf{y}}{\partial x_1} & \dots  & \frac{\partial \mathbf{y}}{\partial x_n}
\end{bmatrix} = \begin{bmatrix} 
\frac{\partial {y_1}}{\partial x_1} & \dots & \frac{\partial {y_1}}{\partial x_n} \\
\vdots & \ddots & \vdots\\
\frac{\partial {y_m}}{\partial x_1} & \dots & \frac{\partial {y_m}}{\partial x_n} \\
\end{bmatrix}
$$
In general, for example, we have a function $g(\mathbf{y}(\mathbf{x}))$. Suppose a vector $\mathbf{v}$ happens to be the gradient of g with respect the vector $\mathbf{y}$ as follows: 
$$
\mathbf{v} = \begin{bmatrix}
\frac{\partial g}{\partial y_1} & \dots & \frac{\partial g}{\partial y_m}
\end{bmatrix} ^T
$$
To get the gradient of g with respect to $\mathbf{x}$, we multiply Jacobian matrix $\mathbf{J}$ with vector $\mathbf{v}$: 

$$\mathbf{J} \cdot \mathbf{v} = \begin{bmatrix} 
\frac{\partial {y_1}}{\partial x_1} & \dots & \frac{\partial {y_m}}{\partial x_n} \\
\vdots & \ddots & \vdots\\
\frac{\partial {y_1}}{\partial x_1} & \dots & \frac{\partial {y_m}}{\partial x_n} \\
\end{bmatrix} 
\begin{bmatrix}
\frac{\partial g}{\partial y_1} \\
\vdots \\ 
\frac{\partial g}{\partial y_m}
\end{bmatrix} = \begin{bmatrix}
\frac{\partial g}{\partial x_1} \\
\vdots \\ 
\frac{\partial g}{\partial x_n}
\end{bmatrix} $$

### Computational graphs 

AD exploits the idea that complicated equations could be converted into a sequence of elementary operations which have specified routines for computing derivatives. This process, also called the evaluation trace, can be visualized by a computational graph where each step is an elementary operation. For example, we want to evaluate the derivative of the function: $$f(x) = 5\exp(x^2)+\sin(3x)$$
Here in this example, the right-most $x_7$ represents the value of $f(x)$, while the left-most $x$ represents our input variable. We construct a computational graph where we take the input $x$ as a node, and we take the constants as nodes as well when applicable. These nodes are connected by lines (edges) to represent the flow of information. 
![](https://i.imgur.com/uBUpnfc.jpg=300x)



### Elementary functions

Elementary Functions | Example | Derivative
:-:|:-:|:-:
exponentials| $e^x$ | $e^x$   
logarithms|$log(x)$| $\frac{1}{x}$ 
powers| $x^2$| $2x$ 
trigonometrics| $sin(x)$ | $cos(x)$ 
inverse trigonometrics|  $arcsin(x)$ | $\frac{1}{\sqrt{(1-x^2)}}$ 


### Dual number
$\forall z_1=x_1+y_1\epsilon, z_2=x_2+y_2\epsilon,$ where $x_1, y_1, x_2, y_2\in \mathbb{R}$, we have the following properties for dual number:
1. $z_1+z_2=(x_1+x_2)+(y_1+y_2)\epsilon$
2. $z_1z_2=(x_1x_2)+(x_1y_2+x_2y_1)\epsilon$
3. $z_1/z_2=(\frac{x_1}{x_2})+\frac{x_2y_1-x_1y_2}{x_2^2}$

As can be seen from the equations above, there is a close connection between the multiplication/division of dual numbers and the product/quotient rules for derivatives:
$$(f(x)g(x))'=f'(x)g(x)+f(x)g'(x)$$
$$(\frac{f(x)}{g(x)})'=\frac{f'(x)g(x)-f(x)g'(x)}{g^{2}(x)}$$.

### Forward mode
Our veritorch package supports forward mode, which uses chain rule to propogate the gradient with respect to the input/independent variables along the computational graph. In the process of gradient propogation, we use the class named Variable that can be updated in a fashion similar to the dual number formula listed above to store the intermediate values and derivatives. In the end, our package will return the jacobian vector product $Jp$. As a result, the forward mode depends on the number of independent parameters involved in the function $f$. If a function actually involves many independent parameters, the forward mode might not be efficient enough and the reverse mode should be considered instead. However, due to time constraint, we might not be able to add support of reverse mode to our veritorch package.

### Reverse mode
Our veritorch package also supports reverse mode, which is still based on the chain rule, but we traverse the chain rule from outside to inside, or, in terms of the computational graph, from right to left. For example, if we have a final node $z$ and input nodes $t_i$, and the path from $z$ to $t_i$ goes through intermediate nodes $x_{p}$, that is, $z = g(x_{p}) = g(f(t_i))$, then we have the drivative:

$$\frac{\partial z}{\partial t_i} = \sum_{p\in parents(i)} \frac{\partial z}{\partial x_p} \frac{\partial x_p}{\partial t_i}$$

That is to say, we only need to know the derivatives of its parents and the formula, in order to calculate the derivative of the output $z$ with respect to input variable $t_i$. More specifically, let's continue to take the example of the function $f(x) = 5\exp(x^2)+\sin(3x)$. The reverse mode starts from the end, where we have a "seed":
$$\frac{\partial f}{\partial x_7} = 1$$

Since we have $x_7 = x_4 + x_6$ and use chain rule, we get:

$$\frac{\partial f}{\partial x_6} = \frac{\partial f}{\partial x_7} \frac{\partial x_7}{\partial x_6} = \frac{\partial f}{\partial x_7}\cdot1 = 1$$

Similarly, given $x_6 = sin(x_5)$, we can easily obtain: 

$$\frac{\partial f}{\partial x_5} = \frac{\partial f}{\partial x_6} \frac{\partial x_6}{\partial x_5} = 1\cdot cosx_5 = cos(3)$$

We do the same thing to all nodes in the computational graph as listed below, and $\frac{\partial f}{\partial x_1}$ is the final answer that we want, as $x_1$ is our input variable. 

$$\frac{\partial f}{\partial x_4} = \frac{\partial f}{\partial x_7} \frac{\partial x_7}{\partial x_4} = 1\cdot 1 = 1$$

$$\frac{\partial f}{\partial x_3} = \frac{\partial f}{\partial x_4} \frac{\partial x_4}{\partial x_3} = 1\cdot 5 = 5$$

$$\frac{\partial f}{\partial x_2} = \frac{\partial f}{\partial x_3} \frac{\partial x_3}{\partial x_2} = 5\cdot exp(x_2^2) = 5e$$

$$\frac{\partial f}{\partial x_1} = \frac{\partial f}{\partial x_2} \frac{\partial x_2}{\partial x_1} + \frac{\partial f}{\partial x_5} \frac{\partial x_5}{\partial x_1}= 5exp(x_2^2)\cdot 2x_1 + cosx_5\cdot3 = 10e + 3cos(3)$$

The table below lists all the basic functions and values that we need for the calculation.

Trace | Elementary Functions | Current Function Value| Derivative wrt 1st Element | Derivative Value wrt 1st Element|  Derivative wrt 2nd Element| Derivative Value wrt 2nd Element
:-:|:-:|:-:|:-:|:-:|:-:|:-:
$x_1$| $x$ | $1$  | $1$| $1$| $1$| $1$  
$x_2$| $x_1^2$ | $1$  | $2x_1$| $2$| $-$| $-$
$x_3$| $exp(x_2)$ | $e$  | $exp(x_2)$| $e$| $-$| $-$
$x_4$| $5x_3$ | $5e$  | $5$| $5$| $-$| $-$
$x_5$| $3x_1$ | $3$  | $3$| $3$| $-$| $-$
$x_6$| $sin(x_5)$ | $sin(3)$  | $cos(x_5)$| $cos(3)$| $-$| $-$
$x_7$| $x_4+x_6$ | $5e+sin(3)$  | $1$| $1$| $1$| $1$

## How to use veritorch

To use the veritorch package, users should first run the commands provided below to install our package via pip and import it. We have already uploaded our veritorch package to PyPI.

```
pip install veritorch
python
>>>import veritorch.veritorch as vt
```

After successfully installing and importing the veritorch package, users can take the following steps to evaluate the derivative of $f$ at a point $x$. Here we take the most complicated case of all: a vector to vector function, as an example: $$\textbf{f(z)} = [f_1(x,y), f_2(x,y)]=[x*y, x*y + 2*x + 2*y]$$ evaluated at $$\textbf{z}=(x,y)=(1,2).$$ 

```
# First, create an instance of Solver class in the veritorch package
# that tracks how many of independent variables $f$ takes as input.

>>>sol=vt.Solver(2)

# Second, define the function:
# (regardless of type of function, MUST RETURN A LIST)

>>>def f(x,y):
...    return [x*y, x*y + 2*x + 2*y]

# Last, pass in the function and point of evaluation to sol to get the Jacobian:

>>>dx=sol.get_diff(f,[1,2])
>>>print(dx)
np.array([[2,1], [4,3]])

# the default mode is forward mode as above, but user can specify "backward" for parameter mode in get_diff method to use reverse mode:

>>>dx=sol.get_diff(f,[1,2], mode = "backward")
>>>print(dx)
np.array([[2,1], [4,3]])

# if the user would like to get function value IN ADDITION 
# to the derivative at the evaluation point, they can call 
# evaluate_and_get_diff method instead, again with both modes implemented:
# (if it's a scalar function, val will return a scalar)
# (if it's a vector function, val will return a list)

>>>val, dx=sol.evaluate_and_get_diff(f,[1,2], "backward")
>>>print(val)
[2,8]
>>>print(dx)
np.array([[2,1], [4,3]])
```

For advanced users, the veritorch package also supports limited, but sometimes more flexible "under-the-hood" usages. Examples below.

Inputting composite functions that involve operations on elementary and common functions, including but not limited to $sin(x), cosh(x), exp(x), arcsin(x), logistic(x)$:


1. Using forward mode:


```
# Example 1: using operation overload:

# First, create an instance of solver class in the veritorch package that
# tracks how many of independent variables $f$ takes as input.

>>>sol=vt.Solver(3)

# Next, use the method create_variable(initial_value) of solver class
# to create variable x1, x2, x3 with their values initialized to 4, 5, 6 
# (and partial derivatives, with respect to x1, x2, x3 respectively 
# initialized to 1 by default)

>>>x1=sol.create_variable(4)
>>>x2=sol.create_variable(5)
>>>x3=sol.create_variable(6)
>>>f1=x1*x2*x3
>>>print(f1)
Variable(120, [30,24,20])

# It outpus (function value, gradient) at the point of evaluation.
```

```
# Example 2: Using functions implemented in Numpy (can directly use np.function method):

>>>import math
>>>import numpy as np
>>>sol=vt.Solver(1)
>>>x1=sol.create_variable(math.pi)
>>>f1=np.sin(x1)
>>>f2=np.cos(f1)
>>>f3=np.exp(f2)
>>>print(f3)
Variable(2.718281828459045, [3.32893514e-16])
```
```
# Example 3: Using functions not implemented in Numpy (cannot use np.function methods):

>>>import numpy as np
>>>sol=vt.Solver(1)
>>>x1=sol.create_variable(3)
>>>f1=x1.logistic()
>>>print(f1)
Variable(0.9525741268224334, [0.04517666])
```


2. Using reverse mode:


```
# Example 1: using operation overload:

# First, create an instance of solver class in the veritorch package that
# tracks how many of independent variables $f$ takes as input.

>>>sol=vt.Solver(3)

# Next, use the method create_variable_b(initial_value) of solver class
# to create variable x1, x2, x3 with their values initialized to 4, 5, 6

>>>x1=sol.create_variable_b(4)
>>>x2=sol.create_variable_b(5)
>>>x3=sol.create_variable_b(6)
>>>f1=x1*x2*x3
>>>f1.grad_value=1
>>>dx1=x1.grad()
>>>dx2=x2.grad()
>>>dx3=x3.grad()

#dx1 is now df1/dx1
>>>print(dx1)
30

#dx2 is now df1/dx2
>>>print(dx2)
24

#dx3 is now df1/dx3
>>>print(dx3)
20

```

```
# Example 2: Using functions implemented in Numpy (can directly use np.function method):

>>>import math
>>>import numpy as np
>>>sol=vt.Solver(1)
>>>x1=sol.create_variable_b(math.pi)
>>>f1=np.sin(x1)
>>>f2=np.cos(f1)
>>>f3=np.exp(f2)
>>>f3.grad_value=1
>>>dx1=x1.grad()
>>>print(dx1)
3.328935140402784e-16
```

```
# Example 3: Using functions not implemented in Numpy (cannot use np.function methods):

>>>import numpy as np
>>>sol=vt.Solver(1)
>>>x1=sol.create_variable_b(3)
>>>f1=x1.logistic()
>>>f1.grad_value=1
>>>dx1=x1.grad()
>>>print(dx1)
0.04517665973091213
```


## Software organization

### Directory structure

The final repo follow the directory structure below:
```
cs207-FinalProject/
    README.md
    requirements.txt
    LICENSE
    setup.py
    veritorch/
        __init__.py
        veritorch.py
        ...
    test/
        test.py
        ...
    docs/
        milestone1.ipynb
        milestone2.ipynb
        documentation.ipynb
        ...
```

### Modules
We choose to have only one module published: the veritorch module within our veritorch package. The veritorch module includes a Solver class, a Variable class and a Variable_b class:
* The Solver class, once initialized with a number $n$, can provide methods to evaluate the value and derivative of many elementary functions that has exactly n independent variable inputs. Both forward mode and reverse mode are supported. The Solver class will use the Variable class to do the forward mode and the Variable_b class to do the backward mode.
* The Variable class takes as input the initial value $x$, and includes methods to overload basic arithmic operators to support forward mode for the veritorch package. 
* The Variable_b class takes as input the initial value $x$, and includes methods to overload basic arithmic operators to support reverse mode for the veritorch package.

### Testing
All files related to testing will be put in the directory cs207-FinalProject/test/. We use TravisCI to check whether each pull request can actually build and Codecov to check how many lines of code have been tested. We have activated both of them for this repo and included the badges in the README.md file at the root directory.

### Distrubution of the package
We use twine to upload the veritorch package to PyPI (Python Packaging Index) for package distribution. This allows users to search and download packages by keywords using pip and pipenv. We have already done so and the latest version can be viewed here: https://pypi.org/project/veritorch/0.1.0/

## Implementation

### Core data structures
#### List
* We use list in the Solver class to track how many independent variables have been created so far and store copies of these independent variables.
* We use list in the Variable_b class to dynamically track the descendants of this variable in the computational graph for reverse mode.

#### Numpy array
In Variable class, a numpy array with length equal to the number of independent variables (provided by the user when the Solver class object is first created) is used to store the derivative information. 

### Core classes and important attributes
There are three core classes in the veritorch package: the solver class, the variable class and the variable_b class

#### Solver class

The Solver class, as indicated above, once initialized with a number $n$, can evaluate the value and derivative of many elementary functions that has exactly n independent variable inputs using the methods presented in the following paragraphs. 

It has the following attributes:
* n: the number of independent variables the target function $f$ has
* independent_variable_list: a list of independent variables that we have created so far using this solver class

The following methods form the core of this Solver class:
* create_variable(x): return an object of the Variable class with value initialized to x and derivative dx initialized to a numpy array of zeros of shape n. If this is the $i^{th}$ time we call the create_variable(x) method of this solver class, then we will set dx[i] to be 1 before we ruturn the created variable. A copy of this created independent variable will also be added to the independent_variable_list.

* create_variable_b(x): return an object of the Variable_b class with value initialized to x and derivative dx initialized to None. A copy of this created independent variable will also be added to the independent_variable_list.

* get_variable(idx): return the copy of the $i^{th}$ independent variable stored in the independent_variable_list. If the user accidentally overwrites the independent variables he/she created before, this method can be called to retrieve a copy of that independent variable so that the user do not need to start the whole process again. However, we recommend the usage of this method for forward mode only, because we choose to dynamically track variables for reverse mode and accidental overwrite is possible to impact this dynamic tracking process.

* merge(d_list): d_list should be a list of derivatives $[df_1, df_2, ..., df_m]$. This method returns the m by n jacobian matrix of $f=[f_1, f_2, ..., f_m]$.

* evaluate_and_get_diff_forward(f, x): This method returns the value and derivative of f evaluated at x computed using the forward mode. Please note that f **must** be defined to return a list, regardless of the type of function.

* evaluate_and_get_diff_backward(f, x): This method returns the value and derivative of f evaluated at x computed using the backward mode. Please note that f **must** be defined to return a list, regardless of the type of function.

* get_diff(f, x, mode="forward"): This method returns the derivative of f evaluated at x computed using the mode specified by user, which is set to be the forward mode by default. Please note that f **must** be defined to return a list, regardless of the type of function. 

* evaluate_and_get_diff(f, x, mode="forward"): This method returns the value and derivative of f evaluated at x computed using the mode specified by user, which is set to be the forward mode by default. Please note that f **must** be defined to return a list, regardless of the type of function.

#### Variable class
This class takes as inputs the initial value $x$ and derivative $dx$ (optional, set to None by default) and includes methods to overload basic arithmic operators to support the forward mode for the veritorch package.

It has the following attributes:
* x: the current scalar value of this variable
* dx: the current derivative of this variable, which should be a vector of length n, where n is the number of independent variables of the function $f$ whose derivative is being evaluated. Note that since the derivative vectors of variables created by the same Solver class and variables that are arithmetic products of those variables created by the same Solver class always have the same length/shape, we no longer need to figure out a way to determine whether two variable objects are independent or not.

We have implemented in this milestone the following methods for this variable class:
* \_\_str\_\_
* \_\_neg\_\_
* \_\_add\_\_
* \_\_radd\_\_
* \_\_sub\_\_
* \_\_rsub\_\_
* \_\_mul\_\_
* \_\_rmul\_\_
* \_\_truediv\_\_
* \_\_rtruediv\_\_
* \_\_pow\_\_
* \_\_eq\_\_
* \_\_ne\_\_

The methods above(excluding the str, eq and ne methods) take as input two variable class objects and return a new variable class object with its value and derivative updated according to the arithmetic rule it corresponds to using the chain rule.

#### Variable_b class
This class takes as inputs the initial value $x$ and includes methods to overload basic arithmic operators to support the reverse mode for the veritorch package.

It has the following attributes:
* value: the current scalar value of this variable
* children: a list, tracking the descendants of this variable in the computational graph and the associated weights fully determined by the chain rule
* grad_value: the current derivative of this variable. It is initialized to None when an object of class Variable_b is created and will remain to be None until the grad() method(presented below) of this node or its ancestors is called. Then grad_value will be updated according to some update rules presented below.

The following method forms the core of this Variable_b class:
* grad(): A recursive method. If the grad_value attribute is not None, this method will simply return the value of grad_value attribute. Otherwise, it will be updated as a weighted sum of the result of the grad() method called on its descendants(stored in the children list), where the weight is fully determined by the chain rule. Please note that since we update the value of grad_value attribute after we finish computing this weighted sum, we will not need to compute the derivative of this variable and its descendants any more if some ancestors of this variable call grad() in the future.

We have implemented the following methods for this Variable_b class:
* \_\_str\_\_
* \_\_neg\_\_
* \_\_add\_\_
* \_\_radd\_\_
* \_\_sub\_\_
* \_\_rsub\_\_
* \_\_mul\_\_
* \_\_rmul\_\_
* \_\_truediv\_\_
* \_\_rtruediv\_\_
* \_\_pow\_\_
* \_\_eq\_\_
* \_\_ne\_\_

The methods above(excluding the str, eq and ne methods) take as input two Variable_b class objects, create a new Variable_b class object with its value and derivative initialized according to the corresponding operation, add the computed weight and this newly created object as a tuple to the children attribute of two input Variable_b class objects and return the newly created object.

### Elementary and common functions
To support the usage of elementary and common functions in our library, we have implemented the following methods for our variable class:
* exp(x)
* log(x)
* sin(x)
* cos(x)
* tan(x)
* arcsin(x)
* arccos(x)
* arctan(x)
* exponential(x, a)
* sinh(x)
* cosh(x)
* tanh(x)
* logistic(x)
* logarithm(x, a)
* sqrt(x)

For the forward mode, the methods above take as input a variable class object and return a new variable class object with its value and derivative updated according to the elementary function it corresponds to using the chain rule. 

For the reverse mode, The methods above take as input two Variable_b class objects, create a new Variable_b class object with its value and derivative initialized according to the corresponding operation, add the computed weight and this newly created object as a tuple to the children attribute of two input Variable_b class objects and return the newly created object.

Special Note:
Usage of the following functions requires some special handling:
* exponential(x, a)
* sinh(x)
* cosh(x)
* tanh(x)
* logistic(x)
* logarithm(x, a)
* sqrt(x)

When we want to define a function, for example, $f(x,y)=($sqrt$($logistic$(x))$, cosh$(y))$ that involves the usage of functions listed above, in order to use the veritorch package, we have to define it in python as follows:
```
define f(x,y):
    return [(x.logistic()).sqrt(), y.cosh()]
```
Then user can use the evaluate_and_get_diff(f, x, mode="forward") method of the Solver class to get the value and derivative of $f$ evaluated at some point $(x_0,y_0)$.

### External dependencies
1. NumPy: it provides us an efficient way to compute the intermediate multidimentional results and organize value and derivative vectors.

2. pytest: it provides us a systematic way to test every line of code written in the veritorch library

3. setuptools: it is used to package our repo before we distribute it.

4. TravisCI and Codecov: they are our test suites. 

## Our extension
We have implemented the reverse mode as our additional feature. The reverse mode supports the same functionality supported by the forward mode. In addition, we have added two methods in the Solver class: 
* get_diff(f, x, mode="forward")
* evaluate_and_get_diff(f, x, mode="forward")

so users can get any function $f$ and its derivative evaluated at some point more conveniently. We have incorporated the relevant mathematical concepts as well as implementation details in Background and Implementation Details sections.  

## Future
### Optimization methods
There are plenty of unconstrained optimization methods that require the derivative information of a function. These optimization methods can be useful for many area of Science. In the future we could implement some of them: Newton's Method, Broyden–Fletcher–Goldfarb–Shanno (BFGS) method, conjugate gradient (CG) method, etc.

As we implement these methods, we will see whether our implementation can give us the required derivative information fast enough and whether the returned results are user friendly or not, and change our implementation accordingly.